In [ ]:
# read a Axioscan 7 czi image by scene
# tile each scene to 512x512x3 
# name each tile by org image name + tile coordinates
# write as png



In [1]:
import os
import sys
import json
import uuid as uuid
import cv2
import re
import shutil
from operator import itemgetter
import math
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import shapely
import shapely.geometry
from shapely.geometry import Polygon,MultiPolygon,GeometryCollection
from shapely.validation import make_valid
from shapely.geometry import mapping
import geopandas as gpd

import imgfileutils as imf
#import segmentation_tools as sgt
from aicsimageio import AICSImage, imread
from skimage import measure, segmentation
from skimage.measure import regionprops
from skimage.color import label2rgb
import progressbar
from IPython.display import display, HTML
from MightyMosaic import MightyMosaic

import argparse
import glob
import bioformats
import javabridge
from PIL import Image

import csv

In [62]:
# setup directory
root = r'/Users/lovely_shufan/'
project_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2022/0_inference_using_MaskRCNN_2021/'
rawdata_dir = project_dir + r'0_raw_czi/GA_MINE_2022_imaged_by_Isabella_Wilson/'
output_dir = project_dir + r'1_tile_png/GA_MINE_2022_imaged_by_Isabella_Wilson/'

#os.makedirs(rawdata_dir, exist_ok=True)
#os.makedirs(output_dir, exist_ok=True)

# address to training images
trainimg_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/0_Image_Collection/ZEISS Primo Star/Georgia/'

In [65]:
blue_means =[]
green_means = []
red_means = []
channels = [blue_means, green_means, red_means]

for path, dirs, files in os.walk(trainimg_dir, topdown=False):
    for name in files:
        if name.endswith('.jpg'):
            img = cv2.imread(os.path.join(path, name), cv2.IMREAD_COLOR)
            for i in range(0,3,1):
                mean_color = np.mean(img[:,:,i])
                channels[i].append(mean_color)

TypeError: 'NoneType' object is not subscriptable

In [55]:
blue_means =[]
green_means = []
red_means = []
channels = [blue_means, green_means, red_means]

for file in os.listdir(trainimg_dir):
    if file.endswith(".jpg"):
        print(file)
        img = cv2.imread(file, cv2.IMREAD_COLOR)
        for i in range(0,3,1):
            mean_color = np.mean(img[:,:,i])
            channels[i].append(mean_color)
            

Snap-447.jpg


[ WARN:0@18251.916] global loadsave.cpp:248 findDecoder imread_('Snap-447.jpg'): can't open/read file: check file path/integrity


TypeError: 'NoneType' object is not subscriptable

In [32]:
print(os.listdir(rawdata_dir))

['PI52606_1.czi', 'PI52606_2.czi', 'PI52606_3.czi']


In [43]:
testimg = AICSImage(os.path.join(rawdata_dir,'PI52606_1.czi'))
print(testimg.scenes)
print(testimg.shape)

('ScanRegion0', 'ScanRegion1', 'ScanRegion2', 'ScanRegion3', 'ScanRegion4', 'ScanRegion5', 'ScanRegion6', 'ScanRegion7', 'ScanRegion8', 'ScanRegion9', 'ScanRegion10', 'ScanRegion11', 'ScanRegion12', 'ScanRegion13', 'ScanRegion14', 'ScanRegion15', 'ScanRegion16', 'ScanRegion17', 'ScanRegion18', 'ScanRegion19', 'ScanRegion20', 'ScanRegion21', 'ScanRegion22', 'ScanRegion23')
(1, 1, 1, 6920, 16850, 3)


In [49]:
def padImg(img, tilex, tiley):
    '''
    :param img:
    :return padded img:
    
    :rtype ndnumpy.array:
    Objective: output a padded image dividle by tile size
    '''
    y = img.shape[0]
    x = img.shape[1]
    pad_top = tiley - (y % tiley)
    pad_lft = tilex - (x % tilex)
    img = cv2.copyMakeBorder(img,pad_top,0,pad_lft,0,cv2.BORDER_CONSTANT,value=[0,0,0])
    
    return img

def makeTiles(file, scene, img, tilex, tiley, output_dir):
    '''
    :param file: name of axioscan7 image file
    :paramtype str:
    :param scene: name of scene
    :paramtype str:
    :param img: 
    :paramtype ndnumpy.array:
    :param tilesize: size of desired tile 
    :paramtype int:  
    :param outputdir: 
    :paramtype str:

    Objective: write tiled images to output dir
    '''
    for i in range(0,img.shape[0],tiley):
        for j in range(0,img.shape[1],tilex):
            #print(i, " ", j)
            #tile = [[j, i],[j, i+1024], [j+1024, i], [j+1024, i+1024]]
            xmin = j
            xmax = j + tilex
            ymin = i
            ymax = i + tiley
            
            subimg = img[ymin:ymax,xmin:xmax]
            tile_id = file[:-4] + '_' +scene + '_' +str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
            
            # write the tile image to the output directory
            cv2.imwrite(os.path.join(output_dir,tile_id+'.png'),subimg)    

In [51]:
for file in os.listdir(rawdata_dir):
    print(file)
    czifile = AICSImage(os.path.join(rawdata_dir,file))
    for scene in czifile.scenes:
        print(scene)
        czifile.set_scene(scene)
        #print(testimg.dims)
        img = czifile.get_image_data("YXS", T=0,C=0,Z=0) # numpy.ndarray
        #print(img.shape)

        # pad the image to be dividible by tile size
        img = padImg(img, 2560, 1920)
        #print(img.shape)
        
        # crop images to tiles
        tiles = makeTiles(file, scene, img, 2560, 1920, output_dir)
        #print(tiles)

PI52606_1.czi
ScanRegion0
ScanRegion1
ScanRegion2
ScanRegion3
ScanRegion4
ScanRegion5
ScanRegion6
ScanRegion7
ScanRegion8
ScanRegion9
ScanRegion10
ScanRegion11
ScanRegion12
ScanRegion13
ScanRegion14
ScanRegion15
ScanRegion16
ScanRegion17
ScanRegion18
ScanRegion19
ScanRegion20
ScanRegion21
ScanRegion22
ScanRegion23
PI52606_2.czi
ScanRegion0
ScanRegion1
ScanRegion2
ScanRegion3
ScanRegion4
ScanRegion5
ScanRegion6
ScanRegion7
ScanRegion8
ScanRegion9
ScanRegion10
ScanRegion11
ScanRegion12
ScanRegion13
ScanRegion14
ScanRegion15
ScanRegion16
ScanRegion17
ScanRegion18
ScanRegion19
ScanRegion20
ScanRegion21
ScanRegion22
ScanRegion23
ScanRegion24
ScanRegion25
ScanRegion26
ScanRegion27
ScanRegion28
ScanRegion29
PI52606_3.czi
ScanRegion1
ScanRegion2
ScanRegion3
ScanRegion4
ScanRegion5
ScanRegion6
ScanRegion7
ScanRegion8
ScanRegion9
ScanRegion10
ScanRegion11
ScanRegion12
ScanRegion13
ScanRegion14
ScanRegion15
ScanRegion16
ScanRegion17
ScanRegion18
ScanRegion19
ScanRegion20
ScanRegion21
ScanRegion22